In [1]:
import numpy as np
import torch 
from torch import nn,optim
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F

from torch.utils.data import DataLoader
%load_ext autoreload
%autoreload 2
import utils
import time

## Residual Block

In [2]:
class ResidualBlock(nn.Module):

    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.shortcut = nn.Sequential() # 何もなければ何もしない
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, X):
        out = F.relu(self.bn1(self.conv1(X)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(X)
        out = F.relu(out)
        return out 


class PreActBlock(nn.Module):

    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1)

        self.shortcut = nn.Sequential() # 何もなければ何もしない
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride),
            )
    
    def forward(self, X):
        out = self.conv1(F.relu(self.bn1(X)))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.shortcut(X)
        return out

In [3]:
resblock = ResidualBlock(3, 64, stride=2)
preact = PreActBlock(3, 64, stride=2)

In [4]:
resblock 

ResidualBlock(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (shortcut): Sequential(
    (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [5]:
preact

PreActBlock(
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (shortcut): Sequential(
    (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(2, 2))
  )
)

In [6]:
X = torch.randn(1, 3, 32, 32)
output = resblock(X)
output.shape

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([1, 64, 16, 16])

In [7]:
X = torch.randn(1, 3, 32, 32)
output = preact(X)
output.shape

torch.Size([1, 64, 16, 16])

## Residual blockの復習

### Residual block 
- 畳み込み層とrelu層とBatch Norm層とskip connectionと組み合わせたもの

In [8]:
# residual blockの復習
class ResidualBlock(nn.Module):

    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.shortcut = nn.Sequential() # 何もなければ何もしない
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, X):
        out = F.relu(self.bn1(self.conv1(X)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(out)
        out = F.relu(out)
        return out

## Bottleneck構造をスクラッチで実装

In [5]:
# bottelneck構造を実装
class BottelNeck(nn.Module):

    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)

        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.conv3 = nn.Conv2d(out_ch, out_ch*4, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_ch*4)

        self.shortcut = nn.Sequential() # 何もなければ何もしない
        if stride != 1 or in_ch != out_ch*4:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch*4, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch*4)
            )
    
    def forward(self, X):
        out = F.relu(self.bn1(self.conv1(X)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(X)
        out = F.relu(out)
        return out

In [6]:
botteleneck = BottelNeck(256, 64, stride=2)
botteleneck

BottelNeck(
  (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (shortcut): Sequential(
    (0): Conv2d(256, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [7]:
X = torch.randn(1, 256, 32, 32)
output = botteleneck(X)
output.shape

torch.Size([1, 256, 16, 16])

## Inception Module 

In [19]:
class InceptionModule(nn.Module):

    def __init__(self, in_ch, out_ch1, out_ch3, out_ch5, out_ch_pool):
        super().__init__()

        # 1x1
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_ch, out_ch1, kernel_size=1),
            nn.BatchNorm2d(out_ch1),
            nn.ReLU()
        )
        # 3x3
        self.branch2 = nn.Sequential(
            # pointwise
            nn.Conv2d(in_ch, out_ch3, kernel_size=1),
            nn.BatchNorm2d(out_ch3),
            nn.ReLU(),
            # 3x3
            nn.Conv2d(out_ch3, out_ch3, kernel_size=3, padding=1), # 出力サイズを変えないようにpadding=1
            nn.BatchNorm2d(out_ch3),
            nn.ReLU()
        )
        self.branch3 = nn.Sequential(
            # pointwise
            nn.Conv2d(in_ch, out_ch5, kernel_size=1),
            nn.BatchNorm2d(out_ch5),
            nn.ReLU(),
            # 3x3
            nn.Conv2d(out_ch5, out_ch5, kernel_size=5, padding=2), # 出力サイズを変えないようにpadding=2
            nn.BatchNorm2d(out_ch5),
            nn.ReLU()
        )

        # pooling
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_ch, out_ch_pool, kernel_size=1),
            nn.BatchNorm2d(out_ch_pool),
            nn.ReLU()
        )

    def forward(self, X):
        branch1 = self.branch1(X)
        branch2 = self.branch2(X)
        branch3 = self.branch3(X)
        branch4 = self.branch4(X)

        return torch.cat([branch1, branch2, branch3, branch4], dim=1)

In [20]:
X = torch.randn(16, 192, 28, 28)
module = InceptionModule(192, 64, 128, 32, 32)
out = module(X)
print(out.shape)

# torch.Size([1, 6])

torch.Size([16, 256, 28, 28])
